In [4]:
import requests
import pandas as pd

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()

df = pd.DataFrame(data)

df.shape

(230, 5)

In [11]:
df

,Intitulé du poste,Date de publication,lieu,competences,Type de poste
0,[\nData Analyst - Publicité\n],\npostée il y a 21 jours\n,[\nParis\n],"[\nsql\n, \nt-sql\n, \noutils\n]","[\n, \n, Sept Lieues SAS, \n, Cabinet de recru..."
1,[Data analyst - H/F\n],\npostée il y a 1 mois\n,"[\nParis, La Défense\n]","[\nsupport\n, \noutils\n, \nagile\n, \ndesign\...","[\n, \n, EDF France, \n, Client final, \n, \n,..."
2,[Customer Data Analyst (H/F)\n],\npostée il y a 27 jours\n,[\nParis\n],"[\ncrm\n, \ndata quality\n, \nsql\n, \nsas\n, ...","[\n, \n, Hermès, \n, Client final, \n, \n, \nC..."
3,[Master Data Analyst H/F\n],\npostée il y a 21 jours\n,[\nMoussy-le-Vieux\n],"[\nsi\n, \nsupport\n, \nsap\n]","[\n, \n, TEREOS, \n, Client final, \n, \n, \nC..."
4,[Portfolio Data Analyst H/F\n],\npostée il y a 3 jours\n,[\nFrance\n],"[\noutils\n, \nsupport\n, \ndecisionnel\n, \ns...","[\n, \n, Safran-Group, \n, Client final, \n, \..."
...,...,...,...,...,...
225,[SAP Fiori Developer (H/F)\n],\npostée il y a 27 jours\n,[\nSaint-Ouen\n],"[\nsap\n, \nabap\n, \nprogiciel\n, \nsystème\n...","[\n, \n, Groupe ENGIE, \n, Client final, \n, \..."
226,[ENGIE Digital - Architecte Données (H/F)\n],\npostée il y a 3 jours\n,[\nParis\n],"[\ndigital\n, \noutils\n, \nsecurité\n, \nsécu...","[\n, \n, Groupe ENGIE, \n, Client final, \n, \..."
227,[Quality Assurance - Lead QA - Lead Acceptance...,\npostée il y a 2 heures\n,[\nBoulogne-Billancourt\n],"[\ntest\n, \nserveur\n, \njquery\n, \nmobile\n...","[\n, \n, Francaise des jeux, \n, Client final,..."
228,[\nDATA SCIENTIST\n],\npostée il y a 16 jours\n,"[\nNEUILLY SUR SEINE\n, \n38.000,00 € - 50.000...","[\nnosql\n, \nunix\n, \njava\n, \nmachine lear...","[\n, \n, BEEZEN, \n, ESN, \n, \n, \nCDI - temp..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Intitulé du poste    230 non-null    object
 1   Date de publication  230 non-null    object
 2   lieu                 230 non-null    object
 3   competences          230 non-null    object
 4   Type de poste        230 non-null    object
dtypes: object(5)
memory usage: 9.1+ KB


In [6]:
df.describe()

,Intitulé du poste,Date de publication,lieu,competences,Type de poste
count,230,230,230,230,230
unique,203,32,72,203,79
top,[DEVELOPPEUR AGILE-(H/F)\n],\npostée il y a 1 mois\n,[\nParis\n],"[\nagile\n, \nsql\n, \nsupport\n, \npython\n, ...","[\n, \n, Société générale, \n, Client final, \..."
freq,3,24,38,4,35


In [8]:
df.columns

Index(['Intitulé du poste', 'Date de publication', 'lieu', 'competences',
       'Type de poste'],
      dtype='object')

In [10]:
df.count()

Intitulé du poste      230
Date de publication    230
lieu                   230
competences            230
Type de poste          230
dtype: int64